In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel
from tqdm import tqdm
import zipfile

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
model_name = 'facebook/dinov2-base'
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
# ZIP 파일 경로
zip_file_path = "/content/drive/MyDrive/AIKU/CAFE_REC/data.zip"



# ZIP 파일 처리
all_img_features = {}
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내부 파일 목록 확인
    file_list = z.namelist()

    # 'data/' 다음 레벨의 위치 디렉토리 추출
    location_list = [
        name.split('/')[1] for name in file_list
        if name.startswith("data/") and name.count('/') > 2 and "image/" in name
    ]

    # ZIP 내부의 위치 디렉토리 추출
    location_list = list(set(location_list))  # 중복 제거

    for location in location_list:
        print(f"-- process {location} --")
        img_feature_dict = {}  # 위치별 딕셔너리 초기화

        # 위치별 이미지 폴더 순회
        folder_path = f"data/{location}/image/"
        cafe_code_list = [
            name.split('/')[3] for name in z.namelist()
            if name.startswith(folder_path) and name.count('/') == 4
        ]
        for cafe_code in tqdm(cafe_code_list):
            cafe_path = f"{folder_path}{cafe_code}/"
            img_list = [name for name in z.namelist() if name.startswith(cafe_path) and name.endswith('.jpg')]

            for img in img_list:
                with z.open(img) as img_file:
                    image = Image.open(img_file)
                    inputs = processor(images=image, return_tensors="pt").to(device)

                    with torch.no_grad():
                        outputs = model(**inputs)
                        pooler_output = outputs.pooler_output.cpu().numpy()

                    img_feature_dict[os.path.basename(img)] = pooler_output  # 위치별로 저장

                    del inputs
                    del outputs
                    torch.cuda.empty_cache()

        # 위치별 데이터 저장
        all_img_features[location] = img_feature_dict

print("Processing complete!")


-- process jongno --


100%|██████████| 871/871 [01:28<00:00,  9.86it/s]


-- process songpa --


100%|██████████| 706/706 [01:17<00:00,  9.16it/s]


-- process Gangnam --


...

-- process gwanak --


100%|██████████| 833/833 [01:28<00:00,  9.43it/s]


-- process gangdong --


100%|██████████| 824/824 [01:28<00:00,  9.27it/s]

Processing complete!


In [ ]:
import numpy as np

def unique_ID_avg(input_dict):
    """
    input : dict (ex. "ID_1" : [numpy_array])
    output : dict (ex. "ID" : [pooled_numpy_array])
    """

    # cafe_ID로 그룹화
    grouped_data = {}
    for key, value in input_dict.items():
        id_part = key.split("_")[0]  # ID 추출
        if id_part not in grouped_data:
            grouped_data[id_part] = []
        grouped_data[id_part].append(value)

    # 2. 각 그룹의 평균 계산
    averaged_data = {}
    for id_part, arrays in grouped_data.items():
        stacked_arrays = np.stack(arrays)  # 배열을 쌓음
        averaged_data[id_part] = np.mean(stacked_arrays, axis=0)  # 평균 계산

    return averaged_data


In [ ]:
import pickle
location_list = list(all_img_features.keys())

for loc in location_list:
  location_dict = unique_ID_avg(all_img_features[loc])

  # JSON 파일로 저장
  output_path = f"/content/drive/MyDrive/AIKU/CAFE_REC/{loc}_embed_dict.pkl"

  with open(output_path, "wb") as f:
    pickle.dump(location_dict, f)



In [ ]:
with open("/content/drive/MyDrive/AIKU/CAFE_REC/mapo_embed_dict.pkl", "rb") as f:
  data_mapo = pickle.load(f)
